In [29]:
import torch
import torchvision
import numpy as np
import torch.nn as nn

In [15]:
net = nn.Sequential(
    nn.Conv2d(1, 96, kernel_size=11, stride=4),  # 54
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),      # 26

    nn.Conv2d(96, 256, kernel_size=5, padding=2), # 26
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2), # 12

    nn.Conv2d(256, 384, kernel_size=3, padding=1), # 12
    nn.ReLU(),
    nn.Conv2d(384, 384, kernel_size=3, padding=1), # 12
    nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1), # 12
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2), # 5
    
    nn.Flatten(),
    
    nn.Linear(5 * 5 * 256, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    
    nn.Linear(4096, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    
    nn.Linear(4096, 10)
)

In [27]:
X = torch.rand(1, 1, 224, 224)

In [28]:
for layer in net:
    X = layer(X)
    print(layer)
    print("output shape: \t", X.shape)
    print("===" * 10)

Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4))
output shape: 	 torch.Size([1, 96, 54, 54])
ReLU()
output shape: 	 torch.Size([1, 96, 54, 54])
MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
output shape: 	 torch.Size([1, 96, 26, 26])
Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
output shape: 	 torch.Size([1, 256, 26, 26])
ReLU()
output shape: 	 torch.Size([1, 256, 26, 26])
MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
output shape: 	 torch.Size([1, 256, 12, 12])
Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
output shape: 	 torch.Size([1, 384, 12, 12])
ReLU()
output shape: 	 torch.Size([1, 384, 12, 12])
Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
output shape: 	 torch.Size([1, 384, 12, 12])
ReLU()
output shape: 	 torch.Size([1, 384, 12, 12])
Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
output shape: 	 torch.Size([1, 256, 12, 12])
ReLU()
o

In [35]:
import torchvision.transforms as transforms
import torch.optim as optim

In [ ]:
device = torch.device("cuda:0")
net.to(device)

In [70]:
batch_size = 128
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

trainset = torchvision.datasets.FashionMNIST("dataset/fashion-mnist",
                                            transform=transform,
                                            train=True,
                                            download=True)
trainset_loader = torch.utils.data.DataLoader(dataset,
                                              batch_size=batch_size,
                                              shuffle=True,
                                              num_workers=4)

testset = torchvision.datasets.FashionMNIST("dataset/fashion-mnist",
                                            transform=transform,
                                            train=False,
                                            download=True)
testset_loader = torch.utils.data.DataLoader(dataset,
                                             batch_size=batch_size,
                                             shuffle=False,
                                             num_workers=2)

In [51]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [52]:
for epoch in range(10):
    print(f"Epoch {epoch + 1}")
    running_loss = 0.0
    
    for i, data in enumerate(trainset_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i % 2000 == 1999:
            print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print("Finished Training")

Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Finished Training


In [72]:
total_predictions = 0
total_correct = 0

for data in testset_loader:
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    
    outputs = torch.argmax(net(inputs), dim=1)
    check = (outputs - labels) == 0
    
    total_correct += len(check[check == True])
    total_predictions += len(check)

print(f"Correct percentage: {total_correct / total_predictions * 100}%")

Correct percentage: 83.75166666666667%
